<a href="https://colab.research.google.com/github/nilu-ai/Restaurant_Fttx_project/blob/main/Fttx_SAD_HAPPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import os
from ipywidgets import widgets

def create_folder_if_not_exists(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

def take_photo_live(id, quality=0.8):
    folder_name = os.path.join('/content/Data/Test/', str(id))
    create_folder_if_not_exists(folder_name)



    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      const cancel = document.createElement('button');
      cancel.textContent = 'Cancel';
      div.appendChild(capture);
      div.appendChild(cancel);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      let stop = false;
      const images = [];

      // Capture images until cancel is clicked.
      while (!stop) {
        await new Promise((resolve) => {
          capture.onclick = () => {
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            const imageData = canvas.toDataURL('image/jpeg', quality);
            images.push(imageData);
            resolve();
          };
          cancel.onclick = () => {
            stop = true;
            resolve();
          };
        });
      }

      stream.getVideoTracks()[0].stop();
      div.remove();
      return images;
    }
    ''')

    display(js)
    images = eval_js('takePhoto({})'.format(quality))

    filenames = []
    for i, image_data in enumerate(images):
        binary = b64decode(image_data.split(',')[1])
        filename = os.path.join(folder_name, 'photo{}.jpg'.format(i))
        filenames.append(filename)
        with open(filename, 'wb') as f:
            f.write(binary)

    return filenames

id_widget = widgets.Text(description="Enter ID:")
display(id_widget)

def on_button_clicked(b):
    try:
        id = id_widget.value
        filenames = take_photo_live(id)
        print('Saved to:')
        for filename in filenames:
            print(filename)
    except Exception as err:
        print(str(err))

button = widgets.Button(description="Capture")
button.on_click(on_button_clicked)
display(button)


Text(value='', description='Enter ID:')

Button(description='Capture', style=ButtonStyle())

<IPython.core.display.Javascript object>

Saved to:
/content/Data/Test/happy/photo0.jpg
/content/Data/Test/happy/photo1.jpg
/content/Data/Test/happy/photo2.jpg
/content/Data/Test/happy/photo3.jpg
/content/Data/Test/happy/photo4.jpg
/content/Data/Test/happy/photo5.jpg
/content/Data/Test/happy/photo6.jpg
/content/Data/Test/happy/photo7.jpg
/content/Data/Test/happy/photo8.jpg


<IPython.core.display.Javascript object>

Saved to:
/content/Data/Test/sad/photo0.jpg
/content/Data/Test/sad/photo1.jpg
/content/Data/Test/sad/photo2.jpg
/content/Data/Test/sad/photo3.jpg
/content/Data/Test/sad/photo4.jpg
/content/Data/Test/sad/photo5.jpg
/content/Data/Test/sad/photo6.jpg
/content/Data/Test/sad/photo7.jpg


<IPython.core.display.Javascript object>

Saved to:
/content/Data/Test/neutral/photo0.jpg
/content/Data/Test/neutral/photo1.jpg
/content/Data/Test/neutral/photo2.jpg
/content/Data/Test/neutral/photo3.jpg
/content/Data/Test/neutral/photo4.jpg
/content/Data/Test/neutral/photo5.jpg
/content/Data/Test/neutral/photo6.jpg
/content/Data/Test/neutral/photo7.jpg
/content/Data/Test/neutral/photo8.jpg
/content/Data/Test/neutral/photo9.jpg
/content/Data/Test/neutral/photo10.jpg
/content/Data/Test/neutral/photo11.jpg
/content/Data/Test/neutral/photo12.jpg


In [53]:
from google.colab import files

uploaded = files.upload()

KeyboardInterrupt: 

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))
    return model

In [ ]:
!unzip data.zip

In [70]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np

train_dir = '/content/Data/Train'
val_dir = '/content/Data/Test'

num_train = 10  # Assuming 3 categories and 10 images per category
num_val = num_train // 5  # 20% validation split

batch_size = 2  # Adjust batch size based on the number of images per category
num_epoch = 30

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='sparse',
        subset='training')  # Use 'training' subset for training data

validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(48,48),
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='sparse',
        subset='validation')  # Use 'validation' subset for validation data

# Check class indices
print(train_generator.class_indices)  # Ensure the indices match your categories

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))  # Adjust to match the number of categories

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model_info = model.fit(
            train_generator,
            steps_per_epoch=num_train // batch_size,
            epochs=num_epoch,
            validation_data=validation_generator,
            validation_steps=num_val // batch_size)
print("model is saved")
model.save_weights('model.h5')


Found 62 images belonging to 3 classes.
Found 14 images belonging to 3 classes.
{'happy': 0, 'neutral': 1, 'sad': 2}
Epoch 1/30
5/5 [==============================] - 4s 135ms/step - loss: 1.1467 - accuracy: 0.6000 - val_loss: 0.5846 - val_accuracy: 1.0000
Epoch 2/30
5/5 [==============================] - 0s 89ms/step - loss: 1.0017 - accuracy: 0.5000 - val_loss: 0.9441 - val_accuracy: 1.0000
Epoch 3/30
5/5 [==============================] - 0s 85ms/step - loss: 1.1519 - accuracy: 0.5000 - val_loss: 0.9482 - val_accuracy: 0.5000
Epoch 4/30
5/5 [==============================] - 0s 83ms/step - loss: 1.2341 - accuracy: 0.1000 - val_loss: 1.0941 - val_accuracy: 0.0000e+00
Epoch 5/30
5/5 [==============================] - 0s 89ms/step - loss: 1.0927 - accuracy: 0.4000 - val_loss: 1.0810 - val_accuracy: 1.0000
Epoch 6/30
5/5 [==============================] - 0s 84ms/step - loss: 1.0888 - accuracy: 0.5000 - val_loss: 1.1129 - val_accuracy: 0.5000
Epoch 7/30
5/5 [============================

In [60]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://89icl8sgwzr-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [71]:
from flask import Flask, request, jsonify
import cv2
import numpy as np
import base64
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

app = Flask(__name__)

# Prevent OpenCL usage and unnecessary logging messages
cv2.ocl.setUseOpenCL(False)

# Define the model architecture

# Load the pre-trained model weights
model = create_model()
model.load_weights('/content/model.h5')


emotion_dict = {0: "Happy", 1: "Neutral", 2: "Sad"}

# Load face detection cascade
face_cascade_path = '/content/haarcascade_frontalface_default.xml'
face_cascade = cv2.CascadeClassifier(face_cascade_path)


if face_cascade.empty():
    raise IOError("Could not load face cascade. Check the path to haarcascade_frontalface_default.xml.")

def detect_and_extract_faces(image, face_cascade):
    faces = face_cascade.detectMultiScale(image, scaleFactor=1.3, minNeighbors=5)
    face_images = []
    coordinates = []
    for (x, y, w, h) in faces:
        face_img = image[y:y+h, x:x+w]
        face_images.append(face_img)
        coordinates.append((x, y, w, h))
    return face_images, coordinates

def predict_emotion(face_img):
    gray_face = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    resized_face = cv2.resize(gray_face, (48, 48))
    normalized_face = np.expand_dims(np.expand_dims(resized_face, -1), 0) / 255.0
    prediction = model.predict(normalized_face)
    max_index = int(np.argmax(prediction))
    predicted_emotion = emotion_dict[max_index]
    confidence = float(prediction[0][max_index])
    return predicted_emotion, confidence

@app.route('/', methods=['GET', 'POST'])
def upload_image():
    if request.method == 'POST':
        data = request.json
        img_data = data['image'].split(',')[1]
        nparr = np.frombuffer(base64.b64decode(img_data), np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        face_images, coordinates = detect_and_extract_faces(img, face_cascade)
        predictions = []
        for face_img, (x, y, w, h) in zip(face_images, coordinates):
            label, confidence = predict_emotion(face_img)
            predictions.append({
                'label': label,
                'confidence': confidence,
                'box': {
                    'x': int(x),
                    'y': int(y),
                    'w': int(w),
                    'h': int(h)
                }
            })
        return jsonify(predictions)
    return """
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
    </head>
    <body>
        <video id="video" width="640" height="480" autoplay></video>
        <canvas id="canvas" width="640" height="480" style="display:none;"></canvas>
        <div id="prediction"></div>
        <script>
            const video = document.getElementById('video');
            const canvas = document.getElementById('canvas');
            const predictionDiv = document.getElementById('prediction');

            async function initCamera() {
                try {
                    const stream = await navigator.mediaDevices.getUserMedia({ video: true });
                    video.srcObject = stream;
                } catch (err) {
                    console.error('Error accessing camera:', err);
                }
            }

            async function captureAndPredict() {
                const context = canvas.getContext('2d');
                context.drawImage(video, 0, 0, canvas.width, canvas.height);
                const imageData = canvas.toDataURL('image/jpeg');
                const response = await fetch('/', {
                    method: 'POST',
                    body: JSON.stringify({ image: imageData }),
                    headers: {
                        'Content-Type': 'application/json'
                    }
                });
                const result = await response.json();
                predictionDiv.innerHTML = JSON.stringify(result, null, 2);
            }

            setInterval(captureAndPredict, 1000);

            initCamera();
        </script>
    </body>
    </html>
    """

if __name__ == '__main__':
    app.run(host='0.0.0.0')


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:10:33] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:10:34] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:10:35] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:10:36] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:10:37] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:10:38] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:10:39] "POST / HTTP/1.1" 200 -


1/1 [==============================] - 0s 93ms/step


INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:10:40] "POST / HTTP/1.1" 200 -


1/1 [==============================] - 0s 24ms/step


...

INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:11:22] "POST / HTTP/1.1" 200 -


1/1 [==============================] - 0s 23ms/step


INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:11:23] "POST / HTTP/1.1" 200 -


1/1 [==============================] - 0s 28ms/step


INFO:werkzeug:127.0.0.1 - - [26/May/2024 06:11:24] "POST / HTTP/1.1" 200 -
